Instead of the U-Net, we will build a Transformer Encoder model. The key difference from first Transformer (which predicted BP) is how we handle the output.

BP Model (Sequence-to-Value): Input (250, 2) -> Transformer Blocks -> GlobalAveragePooling -> Dense(2)

ECG Model (Sequence-to-Sequence): Input (256, 1) -> Transformer Blocks -> TimeDistributed(Dense(1))

We will remove the GlobalAveragePooling1D layer. This way, the model outputs a full sequence, not just a single value. We'll use the data processing functions from the U-Net code, as they are already set up for this task.

Here is the complete code to train a Transformer for PPG-to-ECG synthesis.

In [ ]:
# === 1. Install and Import Libraries ===

# Install xlrd if not already present
!pip install -q xlrd

import os
import zipfile
import glob
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import warnings
import shutil

# Sklearn for metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error

# TensorFlow/Keras for Deep Learning
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Add
from tensorflow.keras.layers import LayerNormalization, Embedding
from tensorflow.keras.layers import MultiHeadAttention, TimeDistributed, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping

# Suppress warnings
warnings.filterwarnings('ignore')

# === 2. Mount Drive & Define Data Functions ===

from google.colab import drive
drive.mount('/content/drive')

def unzip_data(zip_path, extract_folder):
    """Unzips a file and returns a list of all .csv files inside."""
    if not os.path.exists(zip_path):
        print(f"Error: {zip_path} not found. Check your Google Drive path.")
        return []
    os.makedirs(extract_folder, exist_ok=True)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
    csv_files = glob.glob(os.path.join(extract_folder, '**/*.csv'), recursive=True)
    print(f"Extracted {len(csv_files)} files from {zip_path}")
    return csv_files

def create_sequences_ppg_to_ecg(df, seq_length=256, step=128):
    """
    Creates overlapping sequences for PPG-to-ECG translation.
    Input (X) is PPG signal.
    Output (y) is ECG signal.
    """
    ecg = df['ECG'].values
    ppg = df['PPG'].values

    # Normalize signals individually
    ecg_mean, ecg_std = np.mean(ecg), np.std(ecg) + 1e-6
    ppg_mean, ppg_std = np.mean(ppg), np.std(ppg) + 1e-6

    ecg = (ecg - ecg_mean) / ecg_std
    ppg = (ppg - ppg_mean) / ppg_std

    X_seq = []
    y_seq = []

    for i in range(0, len(df) - seq_length, step):
        end_idx = i + seq_length

        X_window = ppg[i:end_idx]
        y_window = ecg[i:end_idx]

        if np.std(X_window) > 0.1 and np.std(y_window) > 0.1:
            X_seq.append(X_window)
            y_seq.append(y_window)

    # Add a "channels" dimension for Conv1D/Transformer
    return np.expand_dims(np.array(X_seq), -1), np.expand_dims(np.array(y_seq), -1)

def load_and_process(zip_path, extract_folder, seq_length=256, debug_limit=None):
    """Main function to load zips and process all files for sequence models."""
    file_list = unzip_data(zip_path, extract_folder)
    if debug_limit is not None:
        file_list = file_list[:debug_limit]
        print(f"--- DEBUG MODE: Processing only {len(file_list)} files. ---")

    if not file_list: return np.array([]), np.array([])
    all_X, all_y = [], []

    for f in tqdm(file_list, desc=f"Processing {zip_path}"):
        try:
            df = pd.read_csv(f)
        except Exception as e:
            print(f"Could not read {f}: {e}")
            continue
        if not all(col in df.columns for col in ['t_sec', 'ECG', 'PPG', 'ABP']):
            print(f"Skipping {f}: missing required columns.")
            continue

        X, y = create_sequences_ppg_to_ecg(df, seq_length=seq_length)
        if X.shape[0] > 0:
            all_X.append(X)
            all_y.append(y)

    if not all_X:
        print(f"No valid data found in {zip_path} for sequence mode.")
        return np.array([]), np.array([])

    all_X = np.concatenate(all_X, axis=0)
    all_y = np.concatenate(all_y, axis=0)
    print(f"Finished processing {zip_path}. Found {all_X.shape[0]} samples.")
    return all_X, all_y

# === 3. Transformer Model Definition ===

def transformer_encoder_block(inputs, head_size, num_heads, ff_dim, dropout=0):
    """Creates a single Transformer encoder block."""
    # Attention and Normalization
    x = LayerNormalization(epsilon=1e-6)(inputs)
    x = MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = Dropout(dropout)(x)
    res = Add()([x, inputs]) # Residual connection

    # Feed-Forward Network and Normalization
    x = LayerNormalization(epsilon=1e-6)(res)
    x = Dense(ff_dim, activation="relu")(x)
    x = Dropout(dropout)(x)
    x = Dense(inputs.shape[-1])(x)
    return Add()([x, res]) # Second residual connection

def build_transformer_seq2seq_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    embed_dim, # We define embedding dim directly
    dropout=0,
):
    """Builds a Transformer-based model for sequence-to-sequence regression."""
    inputs = Input(shape=input_shape) # e.g., (256, 1)
    x = inputs

    # --- 1. Create an "Embedding" for the time-series data ---
    # Project the 1 feature (PPG) into a higher-dimensional space (embed_dim)
    x = Dense(embed_dim)(x)

    # --- 2. Positional Encoding ---
    # We add a simple learned positional embedding.
    positions = tf.range(start=0, limit=input_shape[0], delta=1)
    position_embedding = Embedding(input_dim=input_shape[0], output_dim=embed_dim)(positions)
    x = x + position_embedding

    # --- 3. Create Transformer Blocks ---
    for _ in range(num_transformer_blocks):
        x = transformer_encoder_block(x, head_size, num_heads, ff_dim, dropout)

    # --- 4. Final Head for Sequence Output ---
    # We DON'T pool. Instead, we apply a Dense layer to *every time step*.
    # This projects the embed_dim back down to 1 (our ECG signal).
    outputs = TimeDistributed(Dense(1, activation="linear"))(x)

    return Model(inputs, outputs)


# === 4. Transformer Model Training and Evaluation ===

print("\n--- Starting Transformer PPG-to-ECG Model ---")

# 1. Define Model Parameters
SEQ_LENGTH = 256  # Power of 2 is good for these models
STEP = 128
NUM_FEATURES = 1  # Input is just PPG
NUM_OUTPUTS = 1   # Output is just ECG
BATCH_SIZE = 64
EPOCHS = 20

# --- Transformer-specific Hyperparameters ---
EMBED_DIM = 64    # Dimension to project PPG signal into
HEAD_SIZE = 64    # Dimension of each attention head
NUM_HEADS = 4
FF_DIM = 128      # Hidden layer size in Feed-Forward network
NUM_BLOCKS = 3    # Number of Transformer blocks
DROPOUT = 0.1
# ---------------------------------------------

# --- Define Paths ---
# !!! EDIT THESE PATHS !!!
train_zip_path = '/content/drive/MyDrive/11785FinalData/train.zip'
val_zip_path = '/content/drive/MyDrive/11785FinalData/val.zip'
test_zip_path = '/content/drive/MyDrive/11785FinalData/test.zip'

# 2. Load and process data
X_train_seq, y_train_seq = load_and_process(train_zip_path, 'data/train', seq_length=SEQ_LENGTH)
X_val_seq, y_val_seq = load_and_process(val_zip_path, 'data/val', seq_length=SEQ_LENGTH)
X_test_seq, y_test_seq = load_and_process(test_zip_path, 'data/test', seq_length=SEQ_LENGTH)

if X_train_seq.shape[0] == 0:
    print("No training data found for sequence-based model. Aborting.")
else:
    print(f"Training data shape: {X_train_seq.shape}")
    print(f"Training labels shape: {y_train_seq.shape}")

    # 3. Build and compile the Transformer model
    input_shape = (SEQ_LENGTH, NUM_FEATURES)

    model = build_transformer_seq2seq_model(
        input_shape,
        head_size=HEAD_SIZE,
        num_heads=NUM_HEADS,
        ff_dim=FF_DIM,
        num_transformer_blocks=NUM_BLOCKS,
        embed_dim=EMBED_DIM,
        dropout=DROPOUT,
    )

    # Use 'mean_squared_error' as the loss for signal regression
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
    model.summary()

    # 4. Train Model
    print("\nTraining Transformer model...")
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    history = model.fit(
        X_train_seq, y_train_seq,
        validation_data=(X_val_seq, y_val_seq),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[early_stopping],
        verbose=1
    )

    # 5. Evaluate on Test Set
    print("\nEvaluating Transformer on test set...")
    # This will return [test_loss, test_mae]
    results = model.evaluate(X_test_seq, y_test_seq, batch_size=BATCH_SIZE)
    test_loss = results[0]
    test_mae = results[1]

    # 6. Report Results
    print("\n--- Transformer Model Test Results ---")
    print(f"Test Set MSE (Loss): {test_loss:.4f}")
    print(f"Test Set MAE:        {test_mae:.4f}")
    print("--------------------------------------")


# === 5. Save a Trained Model to Your Drive ===

# First, create a path to a folder in your Google Drive
save_folder = '/content/drive/My Drive/MyProject'
os.makedirs(save_folder, exist_ok=True)

# Define the full path to save your model file
model_save_path = os.path.join(save_folder, 'transformer_ppg_to_ecg_model.keras')

# Save the model
try:
    model.save(model_save_path)
    print(f"Model successfully saved to: {model_save_path}")
except NameError:
    print("Could not save model. Make sure you have trained the model and it is in a variable named 'model'.")
except Exception as e:
    print(f"An error occurred while saving: {e}")